In [2]:
pip install aif360['all']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 65.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install fairlearn

In [4]:
# Core Python Libraries
import numpy as np
import pandas as pd

# Scikit-Learn: Preprocessing, Modeling, Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# XGBoost
from xgboost import XGBClassifier

# AIF360: Datasets and Metrics
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

# AIF360: Preprocessing Algorithms
from aif360.algorithms.preprocessing import (
    Reweighing,
    DisparateImpactRemover,
    LFR
)

# AIF360: In-processing Algorithms
from aif360.algorithms.inprocessing import (
    MetaFairClassifier,
    GerryFairClassifier,
    PrejudiceRemover,
    ExponentiatedGradientReduction,
    GridSearchReduction,
    ARTClassifier,
    AdversarialDebiasing
)

#AIF360: Post-processing Algorithms
from aif360.algorithms.postprocessing import (
    RejectOptionClassification,
    CalibratedEqOddsPostprocessing,
    EqOddsPostprocessing
)

# TensorFlow for AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()


/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [5]:
#data loading

In [6]:
csv_path='/content/AER_credit_card_data.csv'

In [20]:
def load_and_preprocess_creditcard_csv(csv_path):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    # Load the credit card dataset
    df = pd.read_csv(csv_path)

    # Target column: binary 1/0 (1 = accepted, 0 = rejected)
    df['target'] = df['card'].map({'yes': 1, 'no': 0}) if df['card'].dtype == object else df['card']
    df.drop(columns=['card'], inplace=True)

    # Binary protected attribute: age >= 25 → 1 (privileged), else 0
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    # One-hot encode categorical columns
    df_encoded = pd.get_dummies(df.drop(columns=['target', 'age_binary', 'age']), drop_first=True)

    # Feature matrix and labels
    X = df_encoded
    y = df['target']
    protected = df['age_binary']

    # Standardize numeric features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_scaled, y, protected, test_size=0.3, random_state=42, stratify=y
    )

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [57]:
def load_credit_card_data(csv_path):
    import pandas as pd

    # Load dataset
    df = pd.read_csv(csv_path)

    # Binary target column: 1 = approved, 0 = not approved
    df['target'] = df['card'].map({'yes': 1, 'no': 0}) if df['card'].dtype == object else df['card']
    df.drop(columns=['card'], inplace=True)

    # Protected attribute: age >= 25 is privileged
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    # Raw features (exclude target and protected columns)
    X_raw = df.drop(columns=['target', 'age', 'age_binary'])
    y = df['target'].values
    prot = df['age_binary'].values

    return X_raw, y, prot
def preprocess_data(X_raw, y, prot):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    import numpy as np

    # Categorical and numeric columns
    cat_cols = X_raw.select_dtypes(include='object').columns.tolist()
    num_cols = X_raw.select_dtypes(exclude='object').columns.tolist()

    # Train-test split
    X_raw_train, X_raw_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    # Encode categorical
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_cat = encoder.fit_transform(X_raw_train[cat_cols])
    X_test_cat = encoder.transform(X_raw_test[cat_cols])

    # Extract numeric
    X_train_num = X_raw_train[num_cols].values
    X_test_num = X_raw_test[num_cols].values

    # Combine encoded + numeric
    X_train = np.hstack((X_train_num, X_train_cat))
    X_test = np.hstack((X_test_num, X_test_cat))

    # Normalize
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [63]:
def load_creditcard_data_inprocess(csv_path):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder, StandardScaler

    # Load dataset
    df = pd.read_csv(csv_path)

    # Convert 'card' to binary target
    df['target'] = df['card'].map({'yes': 1, 'no': 0}) if df['card'].dtype == object else df['card']
    df.drop(columns=['card'], inplace=True)

    # Protected attribute: age >= 25 is privileged
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    # Features (drop target, age, and binary protected attribute)
    X_raw = df.drop(columns=['target', 'age', 'age_binary'])
    y = df['target'].values
    prot = df['age_binary'].values

    # Train-test split
    X_train_raw, X_test_raw, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    # Identify categorical vs numerical columns
    categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()
    numeric_cols = X_raw.select_dtypes(exclude='object').columns.tolist()

    # One-hot encode categorical features
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_cat = encoder.fit_transform(X_train_raw[categorical_cols])
    X_test_cat = encoder.transform(X_test_raw[categorical_cols])

    # Numeric features
    X_train_num = X_train_raw[numeric_cols].values
    X_test_num = X_test_raw[numeric_cols].values

    # Combine encoded + numeric
    X_train = np.hstack((X_train_num, X_train_cat))
    X_test = np.hstack((X_test_num, X_test_cat))

    # Standardize all features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [39]:
#Utility Functions

In [40]:
def train_baseline_model(X_train, y_train, sample_weight=None):
    model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
    model.fit(X_train, y_train, sample_weight=sample_weight)
    return model


In [41]:
def evaluate_fairness(y_true, y_pred, prot, X=None):
    df = pd.DataFrame(np.hstack((X, y_true[:, None], prot[:, None])),
                      columns=[f"x{i}" for i in range(X.shape[1])] + ['label', 'protected'])

    dataset_true = BinaryLabelDataset(df=df,
                                      label_names=["label"],
                                      protected_attribute_names=["protected"],
                                      favorable_label=1, unfavorable_label=0)

    pred_dataset = dataset_true.copy()
    pred_dataset.labels = y_pred.reshape(-1, 1)

    metric = ClassificationMetric(dataset_true, pred_dataset,
                                  privileged_groups=[{'protected': 1}],
                                  unprivileged_groups=[{'protected': 0}])

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'disparate_impact': metric.disparate_impact(),
        'statistical_parity_difference': metric.statistical_parity_difference(),
        'equal_opportunity_difference': metric.equal_opportunity_difference()
    }


In [42]:
def evaluate_model(model, X_test, y_test, prot_test):
    y_pred = model.predict(X_test)
    return evaluate_fairness(y_test, y_pred, prot_test, X_test)


In [43]:
#XGboost
X_train, X_test, y_train, y_test, prot_train, prot_test = load_and_preprocess_creditcard_csv(csv_path)
baseline_model = train_baseline_model(X_train, y_train)
baseline_metrics = evaluate_model(baseline_model, X_test, y_test.to_numpy(), prot_test.to_numpy())

baseline_metrics

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:41:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.9671717171717171,
 'disparate_impact': np.float64(0.9952838427947598),
 'statistical_parity_difference': np.float64(-0.003648648648648667),
 'equal_opportunity_difference': np.float64(-0.03678414096916305)}

In [44]:
inprocess_results = {}

In [45]:
##disparate impact

In [46]:
def apply_disparate_impact_remover(X_train, y_train, prot_train, repair_level=1.0):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values
    df['protected'] = prot_train.values

    dataset = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    dir_remover = DisparateImpactRemover(repair_level=repair_level)
    repaired_dataset = dir_remover.fit_transform(dataset)

    X_repaired = pd.DataFrame(repaired_dataset.features)
    y_repaired = pd.Series(repaired_dataset.labels.ravel())
    prot_repaired = pd.Series(repaired_dataset.protected_attributes.ravel())

    return X_repaired, y_repaired, prot_repaired


In [47]:
# Apply Disparate Impact Remover to both train and test sets
X_train_repaired, y_train_repaired, prot_train_repaired = apply_disparate_impact_remover(
    pd.DataFrame(X_train), y_train, prot_train
)

X_test_repaired, y_test_repaired, prot_test_repaired = apply_disparate_impact_remover(
    pd.DataFrame(X_test), y_test, prot_test
)

# Train on repaired training data
fair_model = train_baseline_model(X_train_repaired, y_train_repaired)

# Evaluate on repaired test data
fair_metrics = evaluate_model(
    fair_model,
    X_test_repaired.to_numpy(),
    y_test_repaired.to_numpy(),
    prot_test_repaired.to_numpy()
)
inprocess_results['Disparate_Impact_Remover']=fair_metrics
fair_metrics


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:41:16] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.9696969696969697,
 'disparate_impact': np.float64(1.0261333333333333),
 'statistical_parity_difference': np.float64(0.01986486486486494),
 'equal_opportunity_difference': np.float64(-0.015473568281938355)}

In [48]:
##LFR

In [49]:
def apply_lfr(X_train, y_train, prot_train):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values
    df['protected'] = prot_train.values

    dataset = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    lfr = LFR(unprivileged_groups=[{'protected': 0}],
          privileged_groups=[{'protected': 1}],
          k=10, Ax=0.01, Ay=1.0, Az=0.1, verbose=0)


    lfr.fit(dataset)
    transformed_dataset = lfr.transform(dataset)

    X_transformed = pd.DataFrame(transformed_dataset.features)
    y_transformed = pd.Series(transformed_dataset.labels.ravel())
    prot_transformed = pd.Series(transformed_dataset.protected_attributes.ravel())

    return X_transformed, y_transformed, prot_transformed, lfr


In [50]:
X_train_lfr, y_train_lfr, prot_train_lfr, lfr_model = apply_lfr(pd.DataFrame(X_train), y_train, prot_train)


In [51]:
print("Unique labels in y_train_lfr:", np.unique(y_train_lfr))


Unique labels in y_train_lfr: [0. 1.]


In [52]:
fair_lfr_model = train_baseline_model(X_train_lfr, y_train_lfr)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:41:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [53]:
# Apply LFR transformation on test set
df_test = pd.DataFrame(X_test)
df_test['target'] = y_test.values
df_test['protected'] = prot_test.values

test_dataset = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=df_test,
    label_names=['target'],
    protected_attribute_names=['protected']
)

transformed_test_dataset = lfr_model.transform(test_dataset)

X_test_lfr = pd.DataFrame(transformed_test_dataset.features)
y_test_lfr = pd.Series(transformed_test_dataset.labels.ravel())
prot_test_lfr = pd.Series(transformed_test_dataset.protected_attributes.ravel())

# Evaluate
fair_lfr_metrics = evaluate_model(
    fair_lfr_model,
    X_test_lfr.to_numpy(),
    y_test_lfr.to_numpy(),
    prot_test_lfr.to_numpy()
)
inprocess_results['LFR']=fair_lfr_metrics

fair_lfr_metrics


{'accuracy': 0.9974747474747475,
 'disparate_impact': np.float64(0.9270943396226414),
 'statistical_parity_difference': np.float64(-0.06527027027027033),
 'equal_opportunity_difference': np.float64(0.003759398496240629)}

In [54]:
#reweighing

In [58]:
def apply_reweighing(X_train, y_train, prot_train):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df_train,
        label_names=["label"],
        protected_attribute_names=["protected"]
    )
    RW = Reweighing(unprivileged_groups=[{'protected': 0}], privileged_groups=[{'protected': 1}])
    bld_rw = RW.fit_transform(bld_train)

    return bld_rw.features, bld_rw.labels.ravel(), bld_rw.instance_weights


In [59]:
X_train, X_test, y_train, y_test, prot_train, prot_test = preprocess_data(*load_credit_card_data(csv_path))

X_rw, y_rw, sample_weights = apply_reweighing(X_train, y_train, prot_train)

expected_num_features = X_rw.shape[1]

if X_test.shape[1] < expected_num_features:
    padding = expected_num_features - X_test.shape[1]
    X_test_aligned = np.hstack((X_test, np.zeros((X_test.shape[0], padding))))
elif X_test.shape[1] > expected_num_features:
    X_test_aligned = X_test[:, :expected_num_features]
else:
    X_test_aligned = X_test

model_rw = train_baseline_model(X_rw, y_rw, sample_weight=sample_weights)

reweighing_metric = evaluate_model(model_rw, X_test_aligned, y_test, prot_test)

inprocess_results['reweighing'] = reweighing_metric
reweighing_metric


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:43:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.9696969696969697,
 'disparate_impact': np.float64(0.98235807860262),
 'statistical_parity_difference': np.float64(-0.013648648648648676),
 'equal_opportunity_difference': np.float64(-0.03678414096916305)}

In [60]:
for model, metrics in inprocess_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()


🔹 Disparate_Impact_Remover
  accuracy: 0.9697
  disparate_impact: 1.0261
  statistical_parity_difference: 0.0199
  equal_opportunity_difference: -0.0155

🔹 LFR
  accuracy: 0.9975
  disparate_impact: 0.9271
  statistical_parity_difference: -0.0653
  equal_opportunity_difference: 0.0038

🔹 reweighing
  accuracy: 0.9697
  disparate_impact: 0.9824
  statistical_parity_difference: -0.0136
  equal_opportunity_difference: -0.0368



In [61]:
#Inprocess

In [64]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_creditcard_data_inprocess(csv_path)
results = {}

In [65]:
def train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train, label_names=["label"], protected_attribute_names=["protected"],
                                   favorable_label=1, unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test, label_names=["label"], protected_attribute_names=["protected"],
                                  favorable_label=1, unfavorable_label=0)

    clf = GerryFairClassifier(C=100, printflag=False, gamma=0.005, fairness_def='FP', max_iters=50)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [66]:
results['GerryFair'] = train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test)


In [67]:
def train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    clf = PrejudiceRemover(sensitive_attr="protected", eta=25.0)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [68]:
results['PrejudiceRemover'] = train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test)


In [69]:
def train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    expgrad = ExponentiatedGradientReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    expgrad.fit(bld_train)

    pred = expgrad.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [70]:
results['ExponentiatedGradient'] = train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [71]:
def train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    grid = GridSearchReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    grid.fit(bld_train)

    pred = grid.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [72]:
results['GridSearch'] = train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [73]:
tf.disable_eager_execution()  # Required for TF1 compatibility in AIF360

def train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test):
    # Train data
    tf.reset_default_graph()
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    # Test data
    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    # TensorFlow session
    sess = tf.Session()

    clf = AdversarialDebiasing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        scope_name='adv_debiasing',
        sess=sess,
        num_epochs=50,
        batch_size=64,
        debias=True
    )

    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [74]:
results['AdversarialDebiasing'] = train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.637053; batch adversarial loss: 0.777676
epoch 1; iter: 0; batch classifier loss: 0.478905; batch adversarial loss: 0.796420
epoch 2; iter: 0; batch classifier loss: 0.396052; batch adversarial loss: 0.749408
epoch 3; iter: 0; batch classifier loss: 0.518667; batch adversarial loss: 0.804999
epoch 4; iter: 0; batch classifier loss: 0.390882; batch adversarial loss: 0.769547
epoch 5; iter: 0; batch classifier loss: 0.252213; batch adversarial loss: 0.711296
epoch 6; iter: 0; batch classifier loss: 0.238057; batch adversarial loss: 0.709862
epoch 7; iter: 0; batch classifier loss: 0.311911; batch adversarial loss: 0.739607
epoch 8; iter: 0; batch classifier loss: 0.292733; batch adversarial loss: 0.690118
epoch 9; iter: 0; batch classifier loss: 0.225404; batch adversarial loss: 0.669096
epoch 10; iter: 0; batch classifier loss: 0.248713; batch adversarial loss: 0.682964
epoch 11; iter: 0; batch classifier loss: 0.245129; batch adversarial loss:

In [75]:
print("Fairness Evaluation Across Compatible AIF360 In-processing Algorithms\n")
for model, metrics in results.items():
    print(f"🔹 {model}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()


Fairness Evaluation Across Compatible AIF360 In-processing Algorithms

🔹 GerryFair
  accuracy: 0.8030
  disparate_impact: 0.9833
  statistical_parity_difference: -0.0164
  equal_opportunity_difference: 0.0000

🔹 PrejudiceRemover
  accuracy: 0.9242
  disparate_impact: 0.9740
  statistical_parity_difference: -0.0205
  equal_opportunity_difference: -0.0780

🔹 ExponentiatedGradient
  accuracy: 0.9293
  disparate_impact: 0.9226
  statistical_parity_difference: -0.0604
  equal_opportunity_difference: -0.0817

🔹 GridSearch
  accuracy: 0.9293
  disparate_impact: 0.9482
  statistical_parity_difference: -0.0404
  equal_opportunity_difference: -0.0692

🔹 AdversarialDebiasing
  accuracy: 0.9192
  disparate_impact: 0.8905
  statistical_parity_difference: -0.0873
  equal_opportunity_difference: -0.0986



In [76]:
#postprocessing

In [77]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_creditcard_data_inprocess(csv_path)
post_results = {}

In [78]:
def train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)
    bld_pred.labels = y_pred.reshape(-1, 1)

    roc = RejectOptionClassification(
        unprivileged_groups=[{'protected': 0}],
        privileged_groups=[{'protected': 1}],
        low_class_thresh=0.01, high_class_thresh=0.99,
        num_class_thresh=100, num_ROC_margin=50,
        metric_name="Statistical parity difference",
        metric_ub=0.05, metric_lb=-0.05
    )
    roc = roc.fit(bld_test, bld_pred)
    pred = roc.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [79]:
post_results['RejectOptionClassification'] = train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:44:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [80]:
def train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)

    ceo = CalibratedEqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        cost_constraint="fnr",
        seed=42
    )
    ceo = ceo.fit(bld_test, bld_pred)
    pred = ceo.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [81]:
post_results['CalibratedEqOdds'] = train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:44:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [82]:
def train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.labels = y_pred.reshape(-1, 1)

    eq = EqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}]
    )
    eq = eq.fit(bld_test, bld_pred)
    pred = eq.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [83]:
post_results['EqualizedOdds'] = train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:44:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [84]:
for model, metrics in post_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()


🔹 RejectOptionClassification
  accuracy: 0.9773
  disparate_impact: 1.0133
  statistical_parity_difference: 0.0100
  equal_opportunity_difference: -0.0280

🔹 CalibratedEqOdds
  accuracy: 0.9394
  disparate_impact: 0.9497
  statistical_parity_difference: -0.0408
  equal_opportunity_difference: -0.0368

🔹 EqualizedOdds
  accuracy: 0.9470
  disparate_impact: 1.0313
  statistical_parity_difference: 0.0234
  equal_opportunity_difference: -0.0015

